<h1>2D Ising Model: Monte Carlo Simulations</h1>
<h4>William Pugsley </h4>

In [45]:
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from math import sqrt, cosh, sinh, log, exp, tanh

#this module contains the statistical functions used throughout the simulation
from my_math import *

#Thermodynamic constants
k_B = 1.380649e-23 #Units: J/K

<h2>Algorithm</h2>

The Monte Carlo algorithm we will use (Holmes-Cerfon 6-9) consists of creating a grid of vectors free to spin in the XY plane, this will be realized as a d-dimensional array of 2-dimensional vectors with magntiudes of 1 and pointing in random directions. This algorithm will apply to any dimension d. We will randomly select a spin and calculate the energy change, $\Delta E$ associated with replacing the spin with a new random vector. The new state will be accepted with probability $Prob=W(\sigma',\sigma)$, where $W(\sigma',\sigma)$ satisfies the detailed balance equation (Grant 41); for our purposes this will be either the Glauber rule or the Metropolis rule. 

$W_{Metropolis}(\sigma',\sigma)=1 \; \Delta E \leq 0, \: e^{\frac{-\Delta E}{k_BT}} \; \Delta E \gt 0$

$W_{Glauber}(\sigma',\sigma)=\frac{1}{2}(1-tanh(\frac{\Delta E}{2k_BT}))$

We will do this N times for N total spins, this is one Monte Carlo cycle. Completing N cycles for our N spins will consist of one Monte Carlo simulation (Grant 45). At the end of a simulation we can calculate the total magnetization and the magnetization per spin; with these quantities we can verify our theoretical expectation of $<m>$. Repeating for diferent temperatures should yield the temperature dependence of $<m>$. The correlation function, $<S_iS_{i+r}>$ can also be calculated from the same simulation.

<h2>2-Dimensional XY Model</h2>

The two-dimensional XY model consists of a grid of N sites each containing a spin, $\vec{S_i}$, such that $|\vec{S_i}|=1$ and $\vec{S_i}$ is pointing in a random direction for all i=1,...,N. The energy of a state is defined such that neighboring spins pointing in the same direction as well as spins pointing in the direction of an external field, H, are favored.

$E_{state}=-J\sum_{<i,j>}^{N}\vec{S_j}\cdot\vec{S_{i}} - H\sum_{j=1}^N\vec{S_i}$ 

J is strength of interactions between neighboring spins

<h4>Setup</h4>

In [46]:
def rand_unit_xy():
    """ (int) -> (Vector)
    Generates a 2-dimensional unit vector pointing in a random direction.
    """
    return Vector([random.uniform(-1,1), random.uniform(-1,1)]).unit()

def init_grid_1d(length):
    """ (int) -> (list)
    Creates a list with 'length' entries of random 2D Vectors.
    """
    return [rand_unit_xy() for i in range(length)]

def init_grid_2d(row, col=None):
    """ (int, int) -> (2D list)
    Creates a two-dimensional list with row rows and col columns composed of 2D Vectors.
    """
    if col is None: #by default col==row
        col = row
    
    grid = []
    for i in range(row):
        grid.append(init_grid_1d(col))
    
    return grid

def total_mag_2d(grid):
    """ (list) -> (num)
    Calculates the magnitude of the total magnetization of a 2-dimensional grid of 2D Vectors.
    """
    tot = Vector([0,0]) #total
    for row in grid:
        for spin in row:
            tot += spin
    return tot.magnitude()

def mag_per_spin_2d(grid):
    """ (list) -> (float)
    Calculates the magnitude of the magnetization per spin of a 2-dimensional grid of 2D Vectors.
    """
    return total_mag_2d(grid)/len(grid)

def energy_2d(grid, H=None):
    """ (list, Vector) -> (num)
    Calculates the total energy of a 2-dimensional grid of 2D Vectors with an external field, H,
    represented by another 2D Vector.
    """
    if H is None:
        H = Vector([0,0])
        
    e = 0
    num_row = len(grid)
    num_col = len(grid[0])

    for i in range(num_row): #interaction between spins
        for j in range(num_col):
            e += grid[i][j].dot(grid[(i+1)%num_row][(j+1)%num_col])
            e += grid[i][j].dot(grid[(i+1)%num_row][(j-1)%num_col])
            e += grid[i][j].dot(grid[(i-1)%num_row][(j+1)%num_col])
            e += grid[i][j].dot(grid[(i-1)%num_row][(j-1)%num_col])

    for row in grid:
        for spin in row: #external field interactions
            e += H.dot(spin)
    
    return -e/2

def print_grid_2d(grid):
    """ (list) -> (None)
    Prints a visulation of the 2-dimensional grid given as input.
    """
    s = "|"
    for row in grid:
        for spin in row:
            s += str(spin) + ", "
        s = s[:-2] + "\n"
    print(s[:-2] + "|")

In [52]:
def energy_change_2d(grid, row_idx, col_idx, vec, H=None):
    """ (list, int, int, Vector, Vector) -> (int)
    Calculates the energy change associated with flipping the spin at the [row_idx,col_idx] position to the vec Vector. 
    The change is in units of J. H is the external field also in units of J.
    """
    if H is None:
        H = Vector([0,0])

    num_row = len(grid)
    num_col = len(grid[0])

    # change in E = E_final - E_initial
    E_f = -H.dot(vec)
    E_f += -vec.dot(grid[(row_idx+1)%num_row][(col_idx+1)%num_col])
    E_f += -vec.dot(grid[(row_idx+1)%num_row][(col_idx-1)%num_col])
    E_f += -vec.dot(grid[(row_idx-1)%num_row][(col_idx+1)%num_col])
    E_f += -vec.dot(grid[(row_idx-1)%num_row][(col_idx-1)%num_col])

    E_i = H.dot(grid[row_idx][col_idx])
    E_i += grid[row_idx][col_idx].dot(grid[(row_idx+1)%num_row][(col_idx+1)%num_col])
    E_i += grid[row_idx][col_idx].dot(grid[(row_idx+1)%num_row][(col_idx-1)%num_col])
    E_i += grid[row_idx][col_idx].dot(grid[(row_idx-1)%num_row][(col_idx+1)%num_col])
    E_i += grid[row_idx][col_idx].dot(grid[(row_idx-1)%num_row][(col_idx-1)%num_col])

    return E_f - E_i

#we can use the Glauber rule to calculate the probability of a state transition
def glauber_2d(grid, row_idx, col_idx, vec, temp, H=None):
    """ (list, int, int, Vector, float, Vector) -> (float)
    Calculates the probability that a change of state of a 2-dimensional grid, with energy change given by energy, 
    will occur at a given temperature, temp. The probability is calculated using the Glauber rule. 
    """ 
    energy = energy_change_2d(grid, row_idx, col_idx, vec, H)
    return 0.5*(1 - tanh(0.5*energy/temp))

#we can also use metropolis rule
def metropolis_2d(grid, row_idx, col_idx, vec, temp, H=None):
    """ (list, int, int, Vector, float, Vector) -> (float)
    Calculates the probability that a change of state of a 2-dimensional grid, with energy change given by energy, 
    will occur at a given temperature, temp. The probability is calculated using the Metropolis rule. 
    """
    energy = energy_change_2d(grid, row_idx, col_idx, vec, H)
    if energy <= 0:
        return 1
    return exp(-energy/temp) 


def flip_2d(grid, row_idx, col_idx, temp, H=None, rule="metropolis"):
    """ (list, int, int, Vector, float) -> ()
    Flips the spin in grid at the site given by the indices with a probability given by the Glauber or Metropolis 
    rule at temperature temp.
    """
    vec = rand_unit_xy() #new unit Vector

    if rule == "glauber":
        prob = glauber_2d(grid, row_idx, col_idx, vec, temp, H)
    elif rule == "metropolis":
        prob = metropolis_2d(grid, row_idx, col_idx, vec, temp, H)
    else:
        raise ValueError("The rule argument take \"glauber\" or \"metropolis\" as input.") 

    choice = random.uniform(0, 1)
    
    if choice <= prob: #will flip
        grid[row_idx][col_idx] = vec
    #otherwise will not flip and nothing happens

def cycle_2d(grid, temp, H=None, rule='metropolis'):
    """ (list, float, Vector, str) -> ()
    Will conduct one Monte Carlo cycle over grid at temperature temp. 
    """
    num_row = len(grid)
    num_col = len(grid[0])

    for i in range(num_col*num_row): #test flip at each site
        row_idx = random.randint(0, num_row-1)
        col_idx = random.randint(0, num_col-1)
        flip_2d(grid, row_idx, col_idx, temp, H, rule)

def simulation_2d(grid, temp, H=None, rule='metropolis'):
    """ (list, float, Vector, str) -> ()
    Will conduct one Monte Carlo simulation over grid at temperature temp. 
    """
    size = len(grid)*len(grid[0])
    for i in range(size): #N Monte Carlo cycles for N sites is one simulation
        cycle_2d(grid, temp, H, rule)

def plot_quiver_2d(grid):
    """ (list) -> (None)
    Given a 2D list of Vectors, plots the quiver plot of this XY grid.
    """
    plt.figure()
    
    i = len(grid)-1 #iteration counter
    for row in grid:
        X = [x for x in range(len(row))]
        Y = [i for x in range(len(row))]

        U = []
        V = []
        for spin in row:
            U.append(spin[0])
            V.append(spin[1])
        plt.quiver(X,Y,U,V)
        i -= 1

    plt.show()

<h4>Simulation</h4>

We will create a two-dimensional NxM grid of spin sites and run a total of num_sim Monte Carlo simulations before recording the magnetization per spin. We will do this num_points times, to get that many data points, at each temperature T in temperatues. Increasing num_points will get us a more precise measurement of the magnetization per spin at different temperatures. Increasing num_sim will let our experimental results approach the long-time thermodynamic limit 

In [60]:
def full_sim_2d(N, M, num_points, num_sim, temperatures, H=None, output='mag', rule='metropolis'):
    """ (int, int, int, int, np.array, Vector, str, str) -> (list, list) or (list, list, list, list)
    N: total number of rows
    M: total number of columns
    num_points: number of data points at each temperature
    num_sim: total numer of simulations conducted on a grid
    temperatures: range of temperatues at which we will simulate
    H: the strength of the external field
    output: the variable we are measuring (magnetization per spin, energy per spin, or both) 
    """
    if output == "mag":
        magnetizations = [] #<m> of our simulations at various temperatues
        magnetizations_unc = [] #uncertainties in <m>

        for T in temperatures:
            m_at_t = []
            grid = init_grid_2d(N,M)
            for i in range(num_sim): #we do num_sim simulations for each grid
                cycle_2d(grid, T, H, rule)
            for j in range(num_points): #we do num_points simulations at each temperature for that many data points
                cycle_2d(grid, T, H, rule)
                m_at_t.append(mag_per_spin_2d(grid))
            magnetizations.append(mean(m_at_t))
            magnetizations_unc.append(standard_error(m_at_t))
        return magnetizations, magnetizations_unc
    elif output == "energy":
        energies = [] #<E> of our simulations at various temperatues
        energies_unc = [] #uncertainties in <E>

        for T in temperatures:
            e_at_t = []
            grid = init_grid_2d(N, M)
            for i in range(num_sim): #we do num_sim simulations for each grid
                cycle_2d(grid, T, H, rule)
            for j in range(num_points): #we do num_points simulations at each temperature for that many data points
                cycle_2d(grid, T, H, rule)
                e_at_t.append(energy_2d(grid, H)/len(grid)) #energy per spin
            energies.append(mean(e_at_t))
            energies_unc.append(standard_error(e_at_t))
        return energies, energies_unc
    elif output == "both": #finds the correlation function of the 1D Ising model at the first value in temperatures
        energies = [] #<E> of our simulations at various temperatues
        energies_unc = [] #uncertainties in <E>        T = temperatures[0]
        magnetizations = [] #<m> of our simulations at various temperatues
        magnetizations_unc = [] #uncertainties in <m>

        for T in temperatures:
            e_at_t = []
            m_at_t = []
            grid = init_grid_2d(N, M)
            for i in range(num_sim): #we do num_sim simulations for each grid
                cycle_2d(grid, T, H, rule)
            for j in range(num_points): #we do num_points simulations at each temperature for that many data points
                cycle_2d(grid, T, H, rule)
                m_at_t.append(mag_per_spin_2d(grid)) #mag/spin
                e_at_t.append(energy_2d(grid, H)/len(grid)) #energy per spin
            magnetizations.append(mean(m_at_t))
            magnetizations_unc.append(standard_error(m_at_t))
            energies.append(mean(e_at_t))
            energies_unc.append(standard_error(e_at_t))
        return magnetizations, magnetizations_unc, energies, energies_unc
    raise ValueError("The output paramter must be \"mag\", \"energy\", or \"both\".")

In [64]:
m, mu, e, eu= full_sim_2d(5,5,5,5,[1,2],output="both")